In [1]:
from scraper import WebScraper
from time import sleep
import pandas as pd
from json import dumps
from tqdm import tqdm
from bs4 import BeautifulSoup
from glob import glob

In [2]:
cp = pd.read_excel('./cp_catalogue.xlsx', dtype=str)
cp.drop_duplicates(subset='Código Postal', inplace=True)
cp['Código Postal'] = cp['Código Postal'].str.zfill(5)
already_scraped = glob('./results/rappi/*')
already_scraped = set(a.split('/')[-1].split('.')[0].split('_')[-1] for a in already_scraped)
cp = cp[~cp['Código Postal'].isin(already_scraped)]

direcciones = [f'{c}, Ciudad de México' for c in cp['Código Postal']]

In [6]:
len(pd.read_excel('./cp_catalogue.xlsx', dtype=str)['Código Postal'].unique())

1110

In [3]:
source_cates = dict(
    populares_cerca = "https://www.rappi.com.mx/"
    )

source_cate = "populares_cerca"

In [4]:
for current_address in tqdm(direcciones):
    
    scraper = WebScraper()

    scraper.navigate_to_url(source_cates["populares_cerca"])

    scraper.click_element('//*[@id="rappi-web-toolbar"]/div[2]/div/div/div/span')

    final_address = scraper.enter_address(current_address,
                                        input_xpath="//div[starts-with(@id, 'chakra-modal--body-')]//input",
                                        selection_xpath="//div[starts-with(@id, 'chakra-modal--body-')]//ul/li[1]/button"
                                        )

    # Confirm address
    try:
        scraper.click_element('//*[@id="confirm-address-button"]')

        sleep(2)
        # Save address
        scraper.click_element('//*[@id="save-address-button"]')
    except:
        scraper.close()
        
        with open(f'./results/rappi/error_{current_address.split(',')[0]}.txt', 'w') as f:
            f.write('Error al seleccionar dirección (No data)\n')
            
        continue

    cards_data = scraper.scrape_cards(
        cards_xpath='//*[@id="__next"]/div[3]/div[2]/div/div[2]/div',
        aditional_info={"cate_source": source_cate,
                        "current_address": current_address,
                        "final_address": final_address,
                        "platform": 'rappi'
                        }
    )
    
    
    with open(f'./results/rappi/rappi_{source_cate}_{current_address.split(',')[0]}.json', 'w') as f:
        f.write(dumps(cards_data, indent=4))
    
    soup = BeautifulSoup(scraper.driver.page_source, 'html.parser')
    with open(f'./results/rappi/rappi_{source_cate}_{current_address.split(',')[0]}.html', 'w') as f:
        f.write(soup.prettify()) 
    
    
    scraper.close()
    
    sleep(1)

  0%|          | 0/302 [00:00<?, ?it/s]2025-02-02 19:20:59,788 - INFO - Navegando a la URL: https://www.rappi.com.mx/
2025-02-02 19:21:03,288 - INFO - Elemento en //*[@id="rappi-web-toolbar"]/div[2]/div/div/div/span clickeado correctamente.
2025-02-02 19:21:06,564 - INFO - Elemento en //div[starts-with(@id, 'chakra-modal--body-')]//ul/li[1]/button clickeado correctamente.
2025-02-02 19:21:06,565 - INFO - Dirección ingresada correctamente.
2025-02-02 19:21:08,199 - INFO - Elemento en //*[@id="confirm-address-button"] clickeado correctamente.
2025-02-02 19:21:15,327 - INFO - Elemento en //*[@id="save-address-button"] clickeado correctamente.
2025-02-02 19:21:18,375 - WARNING - No se encontró enlace en una tarjeta.
2025-02-02 19:21:20,730 - WARNING - No se encontró enlace en una tarjeta.
2025-02-02 19:21:20,730 - INFO - Scraping completado. Total de tarjetas: 20
  0%|          | 1/302 [00:23<1:59:25, 23.81s/it]2025-02-02 19:21:22,248 - INFO - Navegando a la URL: https://www.rappi.com.mx/
